# Verification of New Results for Bots Case: Symbolic Expansion
#### Unified Model with Integrated Bot 

James Yu, 14 January 2023

In [1]:
from sympy import *

In [2]:
I = Identity(3)
I

I

In [3]:
K1 = MatrixSymbol("K_1", 3, 3)
K2 = MatrixSymbol("K_2", 3, 3)
K3 = MatrixSymbol("K_3", 3, 3)
K = BlockMatrix([[K1, K2], [K2.T, K3]])
K

Matrix([
[  K_1, K_2],
[K_2.T, K_3]])

In [4]:
An = MatrixSymbol("An", 3, 3)
Az = MatrixSymbol("Az", 3, 3)
O = ZeroMatrix(3, 3)
A = BlockMatrix([[An, Az], [O, I]])
A

Matrix([
[An, Az],
[ 0,  I]])

In [5]:
I_block = BlockMatrix([[I, O], [O, I]])
I_block

Matrix([
[I, 0],
[0, I]])

In [6]:
delta, c = symbols("delta, c")

In [7]:
simplified_term_PROTOTYPE = BlockMatrix([
    [I - Inverse(delta*K1 + c*I) * delta * K1, -Inverse(delta*K1 + c*I)*delta*K2],
    [O, I]
])
simplified_term_PROTOTYPE

Matrix([
[(-delta)*(c*I + delta*K_1)**(-1)*K_1 + I, (-delta)*(c*I + delta*K_1)**(-1)*K_2],
[                                       0,                                    I]])

To simplify subsequent calculations we can observe the following:

$$
-\delta (cI + \delta K_1)^{-1}K_1 + I = -(cI + \delta K_1)^{-1}\delta K_1 + I = -(cI + \delta K_1)^{-1}\delta K_1 + (cI + \delta K_1)^{-1}(cI + \delta K_1) = (cI + \delta K_1)^{-1}(cI + \delta K_1 - \delta K_1) = c(cI + \delta K_1)^{-1}
$$

In [8]:
simplified_term = BlockMatrix([
    [c * Inverse(delta*K1 + c*I), -Inverse(delta*K1 + c*I)*delta*K2],
    [O, I]
])
simplified_term

Matrix([
[c*(c*I + delta*K_1)**(-1), (-delta)*(c*I + delta*K_1)**(-1)*K_2],
[                        0,                                    I]])

In [9]:
RHS = I_block + delta * A.T * K * simplified_term * A
RHS

delta*Matrix([
[An.T, 0],
[Az.T, I]])*Matrix([
[  K_1, K_2],
[K_2.T, K_3]])*Matrix([
[c*(c*I + delta*K_1)**(-1), (-delta)*(c*I + delta*K_1)**(-1)*K_2],
[                        0,                                    I]])*Matrix([
[An, Az],
[ 0,  I]]) + Matrix([
[I, 0],
[0, I]])

In [10]:
block_collapse(RHS)

Matrix([
[      (c*delta)*An.T*K_1*(c*I + delta*K_1)**(-1)*An + I,                               delta*(c*An.T*K_1*(c*I + delta*K_1)**(-1)*Az + (-delta)*An.T*K_1*(c*I + delta*K_1)**(-1)*K_2 + An.T*K_2)],
[(c*delta)*(Az.T*K_1 + K_2.T)*(c*I + delta*K_1)**(-1)*An, delta*(c*(Az.T*K_1 + K_2.T)*(c*I + delta*K_1)**(-1)*Az + (-delta)*(Az.T*K_1 + K_2.T)*(c*I + delta*K_1)**(-1)*K_2 + K_3 + Az.T*K_2) + I]])

The above expression should be equivalent to:

In [11]:
K

Matrix([
[  K_1, K_2],
[K_2.T, K_3]])

The upper left equation is equation 6.

If we take the transpose of the lower left expression, we get $K_2$ is equal to:

In [12]:
((c * delta * (Az.T * K1 + K2.T) * Inverse(c*I+delta*K1) * An).T).subs(K1.T, K1) # account for K1 is symmetric

(c*delta)*An.T*(c*I + delta*K_1)**(-1)*(K_1*Az + K_2)

Suppose we take instead the upper right expression of the block matrix:

In [13]:
delta * (c * An.T * K1 * Inverse(c*I + delta*K1)*Az - delta * An.T*K1 * Inverse(c*I + delta*K1)*K2 + An.T * K2)

delta*(c*An.T*K_1*(c*I + delta*K_1)**(-1)*Az + (-delta)*An.T*K_1*(c*I + delta*K_1)**(-1)*K_2 + An.T*K_2)

Next we push $\delta$ into the expression:

In [14]:
(-delta * An.T * (delta * K1) * Inverse(c * I + delta * K1) * K2 + An.T * (delta * K1) * (c * Inverse(c * I + delta * K1))*Az + An.T * (delta * K2))

delta*An.T*K_2 + (-delta**2)*An.T*K_1*(c*I + delta*K_1)**(-1)*K_2 + (c*delta)*An.T*K_1*(c*I + delta*K_1)**(-1)*Az

Now we group like terms with respect to $K_2$:

In [15]:
delta * An.T * (I - (delta * K1) * Inverse(c * I + delta * K1)) * K2 + An.T * (delta * K1) * (c * Inverse(c * I + delta * K1))*Az

delta*An.T*((-delta)*K_1*(c*I + delta*K_1)**(-1) + I)*K_2 + (c*delta)*An.T*K_1*(c*I + delta*K_1)**(-1)*Az

If we stop here for a moment, we can recognize that this is very similar to equation 7 from the note.

$$
An^T c\delta K_1 (\delta K_1 + cI)^{-1}Az + \delta An^T K_1 (I -\delta (\delta K_1 + cI)^{-1}K_2)
$$

As a matter of fact, the $Az$ term is identical. In the symbolic result, the remaining term expands out to:

$$
\delta An^T K_2 - \delta An^T \delta K_1 (cI + \delta K_1)^{-1} K_2
$$

In equation 7, the remaining term expands out to:

$$
\delta An^T K_1 - \delta An^T \delta K_1 (cI + \delta K_1)^{-1} K_2
$$

so the typo is in the first term of these expanded expressions where $K_1$ should instead be $K_2$. This translates to the expression from the symbolic solver.

We can actually keep going from here. If we apply the substitution from before again:

In [16]:
delta * An.T * (c * Inverse(c * I + delta * K1)) * K2 + An.T * (delta * K1) * (c * Inverse(c * I + delta * K1))*Az

(c*delta)*An.T*(c*I + delta*K_1)**(-1)*K_2 + (c*delta)*An.T*K_1*(c*I + delta*K_1)**(-1)*Az

Finally, we do the following operation sequence:

$$
K_1 (cI + \delta K_1)^{-1} = \frac{1}{\delta}I - \frac{1}{\delta}I + \frac{\delta}{\delta}K_1 (cI + \delta K_1)^{-1}\\
= \frac{1}{\delta}[I-I + \delta K_1 (cI + \delta K_1)^{-1}]\\
= \frac{1}{\delta}[I- (cI + \delta K_1)(cI + \delta K_1)^{-1} + \delta K_1 (cI + \delta K_1)^{-1}]\\
= \frac{1}{\delta}[I- (cI + \delta K_1 - \delta K_1)(cI + \delta K_1)^{-1}]\\
= \frac{1}{\delta}[I- cI(cI + \delta K_1)^{-1}]\\
= \frac{1}{\delta}[I- (cI + \delta K_1)^{-1}cI]\\
= \frac{1}{\delta}[(cI + \delta K_1)^{-1}(cI + \delta K_1)- (cI + \delta K_1)^{-1}cI]\\
= \frac{1}{\delta}[(cI + \delta K_1)^{-1}(cI + \delta K_1 - cI)]\\
= \frac{1}{\delta}[(cI + \delta K_1)^{-1}\delta K_1]\\
= (cI + \delta K_1)^{-1}K_1\\
$$

which if we substitute into the above yields:

In [17]:
delta * An.T * (c * Inverse(c * I + delta * K1)) * K2 + An.T * (c * Inverse(c * I + delta * K1))* (delta * K1) *Az

(c*delta)*An.T*(c*I + delta*K_1)**(-1)*K_1*Az + (c*delta)*An.T*(c*I + delta*K_1)**(-1)*K_2

which simplifies to:

In [18]:
c * delta * An.T * Inverse(c * I + delta * K1) * (K2 + K1*Az)

(c*delta)*An.T*(c*I + delta*K_1)**(-1)*(K_1*Az + K_2)

which is, again, the transpose of the lower left entry of the symbolic block matrix, implying that the symbolic block matrix is correctly symmetric.

We can also validate the limit opinion expression:

In [19]:
block_collapse(simplified_term * A)

Matrix([
[c*(c*I + delta*K_1)**(-1)*An, c*(c*I + delta*K_1)**(-1)*Az + (-delta)*(c*I + delta*K_1)**(-1)*K_2],
[                           0,                                                                   I]])

In [20]:
chi = BlockMatrix([
    [MatrixSymbol("x", 3, 1),],
    [MatrixSymbol("z", 3, 1),]
])
chi

Matrix([
[x],
[z]])

In [21]:
block_collapse(simplified_term * A)

Matrix([
[c*(c*I + delta*K_1)**(-1)*An, c*(c*I + delta*K_1)**(-1)*Az + (-delta)*(c*I + delta*K_1)**(-1)*K_2],
[                           0,                                                                   I]])

In [22]:
block_collapse((simplified_term * A)**2)

Matrix([
[(c*(c*I + delta*K_1)**(-1)*An)**2, c*(c*I + delta*K_1)**(-1)*An*(c*(c*I + delta*K_1)**(-1)*Az + (-delta)*(c*I + delta*K_1)**(-1)*K_2) + c*(c*I + delta*K_1)**(-1)*Az + (-delta)*(c*I + delta*K_1)**(-1)*K_2],
[                                0,                                                                                                                                                                        I]])

In [23]:
block_collapse((simplified_term * A)**3)

Matrix([
[(c*(c*I + delta*K_1)**(-1)*An)**3, c*(c*I + delta*K_1)**(-1)*An*(c*(c*I + delta*K_1)**(-1)*Az + (-delta)*(c*I + delta*K_1)**(-1)*K_2) + c*(c*I + delta*K_1)**(-1)*Az + (-delta)*(c*I + delta*K_1)**(-1)*K_2 + (c*(c*I + delta*K_1)**(-1)*An)**2*(c*(c*I + delta*K_1)**(-1)*Az + (-delta)*(c*I + delta*K_1)**(-1)*K_2)],
[                                0,                                                                                                                                                                                                                                                                                  I]])

From here we can see in the limit we would get the limit expression from the note and subsequently the product of matrix inverses law can be used to get the final expression.